In [2]:
import pandas as pd
import requests
import json
import time
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import mimetypes

In [3]:
load_dotenv('google.env')

True

In [4]:
directory = pd.read_csv('../data/HD2024.csv')

In [4]:
def search_directory(query):
    query = query.lower()
    term_checker = lambda row: query in row.lower() if type(row) == str else False

    search_results = pd.concat([
        directory[directory['INSTNM'].apply(term_checker)],
        directory[directory['IALIAS'].apply(term_checker)]
    ]).drop_duplicates().reset_index(drop=True)

    return search_results

In [5]:
def search_directory_ui(query):
    temp = search_directory(query)
    return temp[['UNITID', 'INSTNM', 'STABBR']].to_dict(orient='records')

In [6]:
def param_string(parameters):
    string = ''
    for key in parameters:
        string += '&' + str(key) + '=' + str(parameters[key])
    return string

def assemble_query(parameters):
    url = f'https://customsearch.googleapis.com/customsearch/v1?cx={os.getenv("ENGINE_ID")}&key={os.getenv("API_KEY")}'
    url += param_string(parameters)
    url = url.replace(' ','+')
    return url

def send_query(query):
    data = requests.get(query)
    time.sleep(1)
    if data.status_code == 200:
        data = data.json()
        if 'items' in data:
            results = pd.DataFrame(data['items'])
            results['Query'] = query
            return results
    else:
        raise ValueError(f'query failed with status: {data.status_code}')

In [7]:
def retrieve_cds(unitid):
    curr_college = directory[directory['UNITID'].apply(lambda x: x == unitid)].reset_index(drop=True)

    query = assemble_query({
        'siteSearch': curr_college['WEBADDR'][0],
        'siteSearchFilter': 'i',
        'q': 'common data set',
        'fileType': 'pdf'
    })

    results = send_query(query)

    return results[['htmlTitle', 'link']].to_dict(orient='records')

In [8]:
def frame_url(url):
    if url is None:
        return ''
    else:
        return f'<a href="{url}">View Document</a>'

In [9]:
def retrieve_propublica_summary(unitid):
    curr_college = directory[directory['UNITID'].apply(lambda x: x == unitid)].reset_index(drop=True)
    ein = curr_college['EIN'][0]

    url = f'https://projects.propublica.org/nonprofits/api/v2/organizations/{ein}.json'

    data = requests.get(url).json()

    df = pd.DataFrame(data['filings_with_data'])

    df = df[['tax_prd_yr', 'totrevenue', 'totfuncexpns', 'totassetsend', 'totliabend', 'pdf_url']].rename({
        'tax_prd_yr': 'Year',
        'totrevenue': 'Total revenue',
        'totfuncexpns': 'Total expenses',
        'totassetsend': 'Total assets, end of year',
        'totliabend': 'Total liabilities, end of year',
        'pdf_url': 'Original Filing'
    }, axis=1)

    df['Original Filing'] = df['Original Filing'].apply(frame_url)

    return df.to_html()

In [10]:
unitid = int('204501')

In [11]:
retrieve_propublica_summary(unitid)

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Year</th>\n      <th>Total revenue</th>\n      <th>Total expenses</th>\n      <th>Total assets, end of year</th>\n      <th>Total liabilities, end of year</th>\n      <th>Original Filing</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>2023</td>\n      <td>343982974</td>\n      <td>309146279</td>\n      <td>1896950617</td>\n      <td>447683177</td>\n      <td></td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>2022</td>\n      <td>421346096</td>\n      <td>286125914</td>\n      <td>1820320492</td>\n      <td>392258087</td>\n      <td>&lt;a href="https://projects.propublica.org/nonprofits/download-filing?path=download990pdf_06_2023_prefixes_31-36%2F340714363_202206_990_2023061221443999.pdf"&gt;View Document&lt;/a&gt;</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>2021</td>\n      <td>275432722</td>\n      <td>251268793</td>\n      <td

In [204]:
retrieve_cds('204501')

[{'htmlTitle': '<b>Common Data Set</b> 2024-2025',
  'link': 'https://www.oberlin.edu/media/34940/download?inline'},
 {'htmlTitle': 'Updated CDS AB 7.10.xlsx - Group',
  'link': 'https://www.oberlin.edu/sites/default/files/content/office/institutional-research/documents/2023-24_oc_cds.pdf'},
 {'htmlTitle': 'RStudio IDE Cheat Sheet',
  'link': 'https://www.oberlin.edu/sites/default/files/content/biography/faculty_docs/Jeff_Witmer/rstudio.pdf'},
 {'htmlTitle': 'Instructions for Form I-9, Employment Eligibility Verification',
  'link': 'https://www.oberlin.edu/sites/default/files/2025-06/I-9instr%20%283%29_1.pdf'},
 {'htmlTitle': 'Automating Reading Digital and Analog Meters using Machine ...',
  'link': 'https://www.oberlin.edu/sites/default/files/content/arts-and-sciences/departments/physics/student_projects/2021/thesis_loubna_el_meddah_el_idrissi.pdf'},
 {'htmlTitle': 'ggplot2-cheatsheet-2.0.pdf - <b>Data</b> Visualization',
  'link': 'https://www.oberlin.edu/sites/default/files/conten

In [231]:
query = assemble_query({
    'siteSearch': 'oberlin.edu',
    'siteSearchFilter': 'i',
    'q': 'common data set'
})

results = send_query(query)

In [233]:
results['link'][0]

'https://www.oberlin.edu/institutional-effectiveness/institutional-research-analytics/common-data-set'

In [220]:
query = assemble_query({
    'siteSearch': 'https://www.reed.edu/ir/cds/cdsindex.html',
    'siteSearchFilter': 'i',
    'q': 'common data set',
    'fileType': 'pdf'
})

results = send_query(query)

In [234]:
url = 'https://www.oberlin.edu/institutional-effectiveness/institutional-research-analytics/common-data-set'

In [235]:
soup = BeautifulSoup(requests.get(url).text)

In [250]:
anchors = soup.select('a')

In [239]:
url = 'https://oberlin.edu/media/20096/download'

In [249]:
def get_content_type(url):
    response = requests.get(url)
    return response.headers['content-type']

In [266]:
def is_pdf_anchor(anchor, base):
    if 'href' in anchor.attrs:
        url = anchor.attrs['href']
        if 'https' not in url:
            url = base + url
        response = requests.get(url)
        if response.status_code == 200:
            return response.headers['content-type'] == 'application/pdf'
    return False

In [258]:
anchors[1].attrs['href']

'#main-content'

In [267]:
[is_pdf_anchor(anchor, 'https://oberlin.edu/') for anchor in anchors]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 Fa